In [53]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [54]:
#!pip install memory_profiler
#!pip install psutil
import pandas as pd
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
import sys
from timeit import default_timer as timer
#%load_ext memory_profiler
#from memory_profiler import profile
#import psutil

In [55]:
class Node:
    potentialValue = 0
    def __init__(self, nodeID, balance, edges):
        self.nodeID = nodeID
        self.balance = balance
        self.edges = edges

    def setBalance(self, balance):
        self.balance = balance

    def updatePotential(self, potentialValue):
        self.potentialValue = potentialValue

    def addEdges(self, *edges):
        for edge in edges:
            if self.nodeID != edge.desNode.nodeID:
                self.edges[edge.desNode.nodeID] = edge

    def printNode(self, printAllEdge = False):
        print("node "+str(self.nodeID) + ", balance: " + str(self.balance) +
              ", potential:"+ str(self.potentialValue))
        if printAllEdge == True:
            for edge in self.edges.values():
                print("destination:" + str(edge.desNode.nodeID) + " residualCapacity: "+ str(edge.residualCapacity)
                      + ", reducedTime: "+ str(edge.reducedTime) + ", time: " + str(edge.time))


In [56]:
class Edge:
    flow = 0
    def __init__(self, desNode, time, capacity):
        self.desNode = desNode
        self.time = time
        self.capacity = capacity
        self.residualCapacity = capacity
        self.reducedTime = time

    def updateResidualCapacity(self, residualCapacity):
        self.residualCapacity = residualCapacity

    def updateReducedTime(self, reducedTime):
        self.reducedTime = reducedTime



In [57]:
class PriorityQueue:

    def __init__(self,element):
        self.queue = [element]

    def add(self, element):
        i = 0

        if self.isEmpty():
            self.queue = [element]
            return

        while i < len(self.queue) and element[1] < self.queue[i][1]: # priority lon hon dung truoc
            i += 1

        self.queue.insert(i, element)

    def pop(self):
        return self.queue.pop()[0]

    def contains(self,element):
        return element in self.queue

    def isEmpty(self):
        return len(self.queue) == 0

In [58]:
def dijkstra(nodes,root):
      costs = []  #save the costs from root to other nodes
      #The list is initialized with tuples, where the first element of each tuple represents the cost, and the second element represents the previous node  in the shortest path.
      for i in range(len(nodes)):
          costs.append((sys.maxsize, root)) # sys.maxsize == infinity

      costs[root] = (0, root) #sets the cost of the root node to zero since the cost to reach the root from itself is zero.
      visited = [False for i in range(len(nodes))]
      # queue contains open nodes to visit
      queue = PriorityQueue((root,0)) #The priority queue is used to keep track of the nodes that need to be visited, with priorities based on the current known costs.

      # update costs while queue is not empty
      while not queue.isEmpty():
          node = queue.pop()
          visited[node] = True

          # expand node and update path costs
          for edge in nodes[node].edges.values():
              if not visited[edge.desNode.nodeID] \
                      and edge.reducedTime + costs[node][0] < costs[edge.desNode.nodeID][0] \
                      and edge.residualCapacity > 0:
                  costs[edge.desNode.nodeID] = (edge.reducedTime + costs[node][0], node)

                  if not queue.contains(edge.desNode.nodeID):
                      queue.add((edge.desNode.nodeID,costs[edge.desNode.nodeID]))

      #print("costs list: " + str(costs))
      return costs

In [59]:
class Graph:
    nodes = {}
    sources = set()
    sinks = set()
    def __init__(self, nodeList = None):
        if nodeList == None: return
        for node in nodeList:
            self.nodes[node.nodeID] = node
            if node.balance > 0:
                self.sources.add(node.nodeID)
            elif node.balance < 0:
                self.sinks.add(node.nodeID)

    def updateReduceTime(self):
        for node in self.nodes.values():
            for edge in node.edges.values():
                if edge.residualCapacity > 0:
                    reducedTime = edge.time - node.potentialValue + edge.desNode.potentialValue
                    edge.updateReducedTime(reducedTime)

    def updateFlow(self ,flow, path):
        for idx in range(len(path)-1):
          presentNode = path[idx]
          afterNode = path[idx + 1]
          edge = self.nodes[presentNode].edges[afterNode]
          edge.residualCapacity = edge.residualCapacity - flow
          edge.flow = edge.flow + flow

          if presentNode in self.nodes[afterNode].edges:
            reverseEdge = self.nodes[afterNode].edges[presentNode]
            reverseEdge.flow = reverseEdge.flow - flow
            reverseEdge.residualCapacity += flow

    def updateBalance(self, root, end, flow):
        self.nodes[root].balance -= flow
        self.nodes[end].balance += flow

    def updateResidualNetwork(self, path):
      for index in range(len(path) - 1):
        presentNode = path[index]
        afterNode = path[index + 1]
        if not (path[index] in self.nodes[path[index + 1]].edges):
          # Add edge
           newEdge = Edge(self.nodes[presentNode], -(self.nodes[presentNode].edges[afterNode].time), self.nodes[presentNode].edges[afterNode].flow)
           newEdge.reducedTime =  -(self.nodes[presentNode].edges[afterNode].reducedTime)
           self.nodes[afterNode].addEdges(newEdge)

    def printGraph(self):
        strSources = "Sources:"
        strSinks = "Sinks:"
        for nodeID in self.sources:
            strSources += str(nodeID) +", "
        for nodeID in self.sinks:
            strSinks +=  str(nodeID) +", "
        print(strSources)
        print(strSinks)
        for node in self.nodes.values():
            node.printNode(True)

    def printFlow(self):
        for node in self.nodes.values():
          for edge in node.edges.values():
            if edge.time > 0:
              print(str(node.nodeID) + "->" + str(edge.desNode.nodeID) + "\t Flow: " +str(edge.flow))

    def findPath(self):
        # apply Dijkstra algorithm
        costs = dijkstra(self.nodes, 0)
        #phai sua la uu tien thang end co duong di ngan nhat toi trc nay la do` bat' ki` end
        end=len(self.nodes) -1
        path = [end]
        # iterative search for predecessors
        while end != 0:
          path.append(costs[end][1])
          end = costs[end][1]

        for i in range(1, len(self.nodes)):
          if costs[i][0] != sys.maxsize:
            self.nodes[i].potentialValue -= costs[i][0]

        if path:
          path.reverse()
          return path

    def succesiveShortestPathAlgorithm(self):
      start=timer()
      while(self.nodes[0].balance != 0 and self.nodes[len(self.nodes) - 1].balance !=0 ):
        path= self.findPath()
        #print(path)

        minCapacity = sys.maxsize
        for i in range(len(path)-1):
          residual = self.nodes[path[i]].edges[path[i+1]].residualCapacity
          if residual < minCapacity:
            minCapacity = residual
        flow = min(abs(self.nodes[path[0]].balance), abs(self.nodes[path[-1]].balance), minCapacity)
        #print("flow %d " %flow)

        self.updateReduceTime()
        self.updateFlow(flow,path)
        self.updateBalance(path[0], path[-1], flow)
        self.updateResidualNetwork(path)
        #self.printGraph()

      cost = 0
      for node in self.nodes.values():
        for edge in node.edges.values():
          if edge.time > 0:
            cost += edge.time * edge.flow

      end=timer()
      print("Minimum cost of succesiveShortestPathAlgorithm: " + str(cost))
      print("Run time of succesiveShortestPathAlgorithm is: "+ str(end - start))
      self.printFlow()
      return end-start

    def inputDataFromExcel(self, filepath):
      nodesList = []
      nodeFile = pd.read_excel(filepath, sheet_name='Node')
      edgeFile = pd.read_excel(filepath, sheet_name='Edge')
      for index, row in nodeFile.iterrows():
        nodesList.append(Node(int(row['ID']), int(row['Balance']), {}))
      for node in nodesList:
            self.nodes[node.nodeID] = node
            if node.balance > 0:
                self.sources.add(node.nodeID)
            elif node.balance < 0:
                self.sinks.add(node.nodeID)
      for index, row in edgeFile.iterrows():
        edge = Edge(self.nodes[row['end_ID']], row['time'], row['capacity'])
        self.nodes[row['start_ID']].addEdges(edge)

    def checkMassBalanceConstraint(self):
      sum=0
      for node in self.nodes.values():
        sum+=node.balance
      return (sum==0)

    def addDummySource(self):
      dummySource = Node(0, 0, {}) # initialize balance =0
      self.nodes[0]= dummySource
      for nodeID in self.sources:
        self.nodes[0].balance+= self.nodes[nodeID].balance
        edge=Edge(self.nodes[nodeID], 0, self.nodes[nodeID].balance)
        self.nodes[0].addEdges(edge)

    def addDummySink(self):
      dummySink = Node(len(self.nodes), 0, {}) # initialize balance =0
      for nodeID in self.sinks:
        dummySink.balance+= self.nodes[nodeID].balance
        edge=Edge(dummySink, 0, -self.nodes[nodeID].balance)
        self.nodes[nodeID].addEdges(edge)
      self.nodes[len(self.nodes)]= dummySink

    def drawGraph(self, path, node_size=1500, node_color='orange', node_alpha=0.3,
                      node_text_size=12, edge_alpha=0.8, edge_tickness=1, edge_text_pos=0.7, text_font='sans-serif',
                      special_edge_color='red', normal_edge_color = 'grey'):
      G = nx.DiGraph()

      normal_edges = []
      path_edges = []

      for node in self.nodes.values():
        G.add_node(node.nodeID)
        for edge in node.edges.values():
          G.add_edge(node.nodeID, edge.desNode.nodeID, capacity=edge.residualCapacity, time=edge.time, flow=edge.flow)
          normal_edges.append((node.nodeID, edge.desNode.nodeID))

      for idx in range(len(path)-1):
        path_edges.append((path[idx], path[idx+1]))
        normal_edges.remove((path[idx], path[idx+1]))

      graph_pos = nx.drawing.nx_pydot.graphviz_layout(G, 'circo')

      nx.draw_networkx_edges(G, graph_pos, edgelist=normal_edges, edge_color='blue',
                          width=edge_tickness, alpha=edge_alpha)

      # Draw special edges with different color
      nx.draw_networkx_edges(G, graph_pos, edgelist=path_edges, edge_color='red',
                            width=edge_tickness, alpha=edge_alpha)

      nx.draw_networkx_nodes(G, graph_pos, node_size=node_size, alpha=node_alpha, node_color=node_color)
      node_labels = {node: str(node) for node in G.nodes()}
      nx.draw_networkx_labels(G, graph_pos, labels=node_labels, font_size=node_text_size, font_family=text_font)
      edge_labels = {(u, v): f"t:{G[u][v]['time']}, f:{G[u][v]['flow']}, c:{G[u][v]['capacity']}"
                   for u, v, d in G.edges(data=True) if 'time' in d and 'flow' in d }

      # Adjust the position of edge labels based on the position of the nodes
      edge_label_pos = {(u, v): (graph_pos[u][0] + graph_pos[v][0]) / 2,
                        (u, v): (graph_pos[u][1] + graph_pos[v][1]) / 2}

      nx.draw_networkx_edge_labels(G, graph_pos, edge_labels=edge_labels, pos=edge_label_pos, font_size=8,
                                verticalalignment="bottom", horizontalalignment="right")

      plt.show()

    def simplexAlgorithm(self):
        G = nx.DiGraph()

        for node in self.nodes.values():
          if(node.balance !=0):
            G.add_node(node.nodeID, demand=-node.balance) #positive balance is sink in networkX and reversely
          else:
            G.add_node(node.nodeID)
          for edge in node.edges.values():
            G.add_edge(node.nodeID, edge.desNode.nodeID, weight= edge.time, capacity= edge.capacity)

        # graph_pos = nx.spectral_layout(G)

        # nx.draw(G, graph_pos, with_labels = True, font_color = 'white', node_shape='s')

        start=timer()
        flowCost, flowDict = nx.network_simplex(G)
        end=timer()
        print('Minimum cost of simplex Algorithm:', flowCost)
        print("Run time of simplex algorithm is: "+ str(end - start))

        for key_i, inner_dict in flowDict.items():
          for key_j, inner_val in inner_dict.items():
            print(f'{key_i}->{key_j} \t Flow: {inner_val}')



        return end-start


In [60]:
# evaluate efficiency purpose
myGraph=Graph()
runTimeSimplex=np.zeros(10)
runTimeSSPA=np.zeros(10)
for i in range (10): #data generate by our source code always satisfy mass balance constraint, so we don't need to check it.
  filepath="/content/drive/MyDrive/MM/data52/mm_"+str(i)+".xlsx"
  myGraph.inputDataFromExcel(filepath)
  simplexRunTime = myGraph.simplexAlgorithm()
  myGraph.addDummySource()
  myGraph.addDummySink()
  succesiveShortestPathAlgorithmRuntime = myGraph.succesiveShortestPathAlgorithm()
  runTimeSimplex[i] =simplexRunTime
  runTimeSSPA[i]=succesiveShortestPathAlgorithmRuntime
  print("\n\n")

averageRunTimeSimplex=0
averagerunTimeSSPA=0
for i in range (10):
  averageRunTimeSimplex+=runTimeSimplex[i]
  averagerunTimeSSPA+=runTimeSSPA[i]

averageRunTimeSimplex/=10
averagerunTimeSSPA/=10
print(str(averageRunTimeSimplex)+"\n")
print(averagerunTimeSSPA)

runTimeSimplex = np.insert(runTimeSimplex, 0, np.nan) # Start with 1
plt.plot(runTimeSimplex, marker='o', linestyle='-')
plt.axhline(y=averageRunTimeSimplex, color='red', linestyle='--', label='Average Value')
plt.xlim(0, len(runTimeSimplex))


plt.annotate(f'{averageRunTimeSimplex:.5f}',
             xy=(len(runTimeSimplex) + 3, averageRunTimeSimplex),
             xytext=(len(runTimeSimplex) + 1, averageRunTimeSimplex),
             ha='center', va='bottom', color='red', fontsize=10,
             bbox=dict(boxstyle='round,pad=0.3', edgecolor='red', facecolor='white'))

plt.legend() # Display caption

plt.xlabel('Iteration')
plt.ylabel('Run Time')
plt.title('Average Run Time Of Simplex Algorithm')

plt.show()

#runTimeSSPA
runTimeSSPA = np.insert(runTimeSSPA, 0, np.nan) # Start with 1
plt.plot(runTimeSSPA, marker='o', linestyle='-')
plt.axhline(y=averagerunTimeSSPA, color='red', linestyle='--', label='Average Value')
plt.xlim(0, len(runTimeSSPA))


plt.annotate(f'{averagerunTimeSSPA:.5f}',
             xy=(len(runTimeSSPA ) + 3, averagerunTimeSSPA),
             xytext=(len(runTimeSSPA ) + 1, averagerunTimeSSPA),
             ha='center', va='bottom', color='red', fontsize=10,
             bbox=dict(boxstyle='round,pad=0.3', edgecolor='red', facecolor='white'))

plt.legend() # Display caption

plt.xlabel('Iteration')
plt.ylabel('Run Time')
plt.title('Average Run Time Of SSPA Algorithm')

plt.show()

In [61]:
########################main###############################
# node(value,balance, { exitEdges(node,capacity,weight})
#testing output purpose

myGraph=Graph()
myGraph.inputDataFromExcel("/content/drive/MyDrive/MM/data.xlsx")
#myGraph.drawGraph([])
#myGraph.addDummySource()
#myGraph.printGraph()
if(myGraph.checkMassBalanceConstraint()==0):
  print("The problem has no solution because of violating mass balance constraint\n")
else:
  #%memit -r 1 memory_intensive_operation()
  myGraph.simplexAlgorithm()
  myGraph.addDummySource()
  myGraph.addDummySink()
  #myGraph.printGraph()
  myGraph.succesiveShortestPathAlgorithm()

Minimum cost of simplex Algorithm: 26
Run time of simplex algorithm is: 0.00037887000007685856
1->2 	 Flow: 1
1->3 	 Flow: 3
2->3 	 Flow: 0
2->4 	 Flow: 1
3->4 	 Flow: 3
Minimum cost of succesiveShortestPathAlgorithm: 26
Run time of succesiveShortestPathAlgorithm is: 0.0002111730000251555
1->2	 Flow: 1
1->3	 Flow: 3
2->3	 Flow: 0
2->4	 Flow: 1
3->4	 Flow: 3
